In [23]:
import pandas as pd
import json
import numpy as np
#from collections import defaultdict
#from collections import Counter
import sklearn.linear_model
from sklearn.metrics import precision_recall_fscore_support as score
#import os
pd.options.mode.chained_assignment = None

In [24]:
### Purpose: get smaller-than-actual versions of each CSV, take a look at what we can get from each

sourceFilePath = "C:\\Users\\zange\\OneDrive\\Documents\\Business\\Education\\Drexel\\CS613\\dota-2-matches"

In [28]:
# Moved to Excel to more easily remove/rename columns,ended up with the following:
match = pd.read_csv(open(sourceFilePath+"\\match.csv"))

In [29]:
match.head()

,match_id,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
0,0,1446750112,2375,1982,4,3,63,1,22,True,0,1,155
1,1,1446753078,2582,0,1846,63,0,221,22,False,0,2,154
2,2,1446764586,2716,256,1972,63,48,190,22,False,0,0,132
3,3,1446765723,3085,4,1924,51,3,40,22,False,0,0,191
4,4,1446796385,1887,2047,0,0,63,58,22,True,0,0,156


In [30]:
players = pd.read_csv(open(sourceFilePath+"\\players.csv"))

In [38]:
# so match_id has the win results and player has the player slot and the hero_id.
# let's pull the hero_id, player_slot,account_id,and match_id from players and see where that gets us
playersT1 = pd.DataFrame(players['match_id'])
playersT1['account_id'] = players['account_id']
playersT1['hero_id'] = players['hero_id']
playersT1['player_slot'] = players['player_slot']

In [40]:
playersT1.head()

,match_id,account_id,hero_id,player_slot
0,0,0,86,0
1,0,1,51,1
2,0,0,83,2
3,0,2,11,3
4,0,3,67,4


In [43]:
# The problem with hero selection is - how do you compare them? There are so SO many types of 5-member teams available, and across the
# 50,000 matches it is wrong to think that players were always picking characters the same way.
# Sometimes players communicate to pick characters. Sometimes they do so randomly. There is not a clear 'this person is playing linebacker'
# kind of thing in DotA2, making comparisons between teams very challenging.

# THAT SAID...

# Characters are labelled based on specific categories. So I'll start with those first.
# here is an enhanced hero_names table that includes additional details on each of the characters.
####***use of the 6 for the skills, in order, ranked, those like Zeus with only one category get all 6 in that one,
####***those with 2 get 4,2 and then three or more 3,2,1
heroExtended = pd.read_csv(open(sourceFilePath+"\\hero_names_categories.csv"))

In [49]:
heroExtended = heroExtended.fillna(0)

In [61]:
heroExtended.keys()

Index(['name', 'hero_id', 'localized_name', 'Carry', 'Disabler',
       'Lane_Support', 'Initiator', 'Jungler', 'Support', 'Durable', 'Nuker',
       'Pusher', 'Escape', 'Melee', 'Ranged', 'Strength', 'Agility',
       'Intelligence'],
      dtype='object')

In [51]:
heroExtended.to_csv(open(sourceFilePath+"\\hero_names_final.csv","w"))

In [58]:
#### Now the goal is to get both teams into the same dataframe not as individual PLAYERS, but instead as the characters themselves by category
#### Start by getting every hero's data appended to the playersT1 dataframe
headers = list(heroExtended.keys())
headers = headers[3:]
headers
for header in headers:
    playersT1[header] = 0

In [59]:
playersT1.keys()

Index(['match_id', 'account_id', 'hero_id', 'player_slot', 'Carry', 'Disabler',
       'Lane_Support', 'Initiator', 'Jungler', 'Support', 'Durable', 'Nuker',
       'Pusher', 'Escape', 'Melee', 'Ranged', 'Strength', 'Agility',
       'Intelligence'],
      dtype='object')

In [74]:
# Error in data: hero_id==0 not defined. No corresponding hero. Will remove these matches from dataset
# Runtime: 2 hours
errorMatches = []
for idx,row in playersT1.iterrows():
    for header in headers:
        if row['hero_id'] != 0:
            playersT1.at[idx,header] = float(heroExtended[heroExtended['hero_id']==row['hero_id']][header])
        else:
            errorMatches.append(row['match_id'])

In [88]:
#playersT1.tail()
#playersT1[playersT1['hero_id']==0]
errorMatchSet = set(errorMatches)
print(len(errorMatchSet))
maskSafeMatches = [id not in errorMatchSet for id in playersT1['match_id']]

35


In [93]:
#Removing everything that has a 0 hero
playersT1Safe = playersT1[maskSafeMatches]

In [94]:
playersT1Safe.to_csv(open(sourceFilePath+"\\playersT1safe_raw.csv","w"))

In [99]:
# same with matches
maskSafeMatches = [id not in errorMatchSet for id in match['match_id']]
matchSafe = match[maskSafeMatches]
matchSafe.to_csv(open(sourceFilePath+"\\matchsafe_raw.csv","w"))

In [104]:
#now to combine a dataframe together
playersT1Safe['player_slot'].unique()

array([  0,   1,   2,   3,   4, 128, 129, 130, 131, 132], dtype=int64)

In [106]:
playersT1Safe.keys()

Index(['match_id', 'account_id', 'hero_id', 'player_slot', 'Carry', 'Disabler',
       'Lane_Support', 'Initiator', 'Jungler', 'Support', 'Durable', 'Nuker',
       'Pusher', 'Escape', 'Melee', 'Ranged', 'Strength', 'Agility',
       'Intelligence'],
      dtype='object')

In [112]:
mask = [val < 128 for val in playersT1Safe['player_slot']]
playersT1Radiant = playersT1Safe[mask]
mask = [val >= 128 for val in playersT1Safe['player_slot']]
playersT1Dire = playersT1Safe[mask]

In [113]:
playersT1Radiant.describe()

,match_id,account_id,hero_id,player_slot,Carry,Disabler,Lane_Support,Initiator,Jungler,Support,Durable,Nuker,Pusher,Escape,Melee,Ranged,Strength,Agility,Intelligence
count,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.0,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000
mean,24999.153387,39415.369463,50.477134,2.000000,1.861567,0.773029,0.0,0.532405,0.081745,1.025962,0.353539,0.772309,0.158163,0.441281,0.501879,0.498121,0.328422,0.308460,0.363118
std,14433.622404,46689.293357,32.844509,1.414216,1.541294,0.877060,0.0,0.937871,0.376550,1.345627,0.685922,1.026444,0.485770,0.825622,0.499997,0.499997,0.469640,0.461858,0.480899
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.000000,0.000000,21.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24996.000000,18963.000000,47.000000,2.000000,3.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,37499.000000,70199.000000,75.000000,3.000000,3.000000,1.000000,0.0,1.000000,0.000000,3.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,49999.000000,158359.000000,112.000000,4.000000,4.000000,3.000000,0.0,3.000000,2.000000,3.000000,3.000000,6.000000,2.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [114]:
playersT1Dire.describe()

,match_id,account_id,hero_id,player_slot,Carry,Disabler,Lane_Support,Initiator,Jungler,Support,Durable,Nuker,Pusher,Escape,Melee,Ranged,Strength,Agility,Intelligence
count,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.0,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000,249825.000000
mean,24999.153387,39765.960813,50.631426,130.000000,1.860606,0.769899,0.0,0.528054,0.085032,1.022115,0.354068,0.774554,0.159836,0.445836,0.501911,0.498089,0.326240,0.305982,0.367777
std,14433.622404,46832.893294,32.769155,1.414216,1.535702,0.874689,0.0,0.934677,0.384535,1.344308,0.687357,1.030401,0.488437,0.829917,0.499997,0.499997,0.468837,0.460823,0.482201
min,0.000000,0.000000,1.000000,128.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.000000,0.000000,21.000000,129.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24996.000000,19481.000000,47.000000,130.000000,3.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,37499.000000,71101.000000,75.000000,131.000000,3.000000,1.000000,0.0,1.000000,0.000000,3.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,49999.000000,158360.000000,112.000000,132.000000,4.000000,3.000000,0.0,3.000000,2.000000,3.000000,3.000000,6.000000,2.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Running this overnight, should do all the sums needed and get it into a saved dataframe.
## Tomorrow can worry about getting this through the regression

radiant = np.array([[99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999,99999]])
dire = np.array([[88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888,88888]])
for match in playersT1Safe['match_id'].unique():
    try:
        mask = [val == match for val in playersT1Radiant['match_id']]
        sliced = playersT1Radiant[mask]
        output = [match]
        output.extend(list(sliced.sum()[4:].values)) # Carry:Intelligence
        output = np.array([output])
        radiant = np.concatenate((radiant,output),axis=0)
        mask = [val == match for val in playersT1Dire['match_id']]
        sliced = playersT1Dire[mask]
        output = [match]
        output.extend(list(sliced.sum()[4:].values))
        output = np.array([output])
        dire = np.concatenate((dire,output),axis=0)
    except:
        print("Encountered an error")
        print(match)
    
combined = np.concatenate((radiant,dire),axis=1)
headers = ['radMatch_id','radCarry','radDisabler','radLane_Support','radInitiator', \
           'radJungler','radSupport','radDurable','radNuker','radPusher','radEscape', \
           'radMelee','radRanged','radStrength','radAgility','radIntelligence', \
           'dirMatch_id','dirCarry','dirDisabler','dirLane_Support','dirInitiator', \
           'dirJungler','dirSupport','dirDurable','dirNuker','dirPusher','dirEscape', \
           'dirMelee','dirRanged','dirStrength','dirAgility','dirIntelligence', \
]
combined = pd.DataFrame(combined,index=None,columns=headers)
combined = combined.iloc[1:]
combined.to_csv(open(sourceFilePath+"\\t1Combined.csv","w"))

In [6]:
###Now that we have restitched everything together, it's time to figure out how best to perform this logistic regression
### Let's pull the four features we want into a numpy array
X = pd.DataFrame(playerSkills['Dire_meanTrueSkillmu'])
X['Dire_meanTrueSkillsigma'] = playerSkills['Dire_meanTrueSkillsigma']
X['Radi_meanTrueSkillmu'] = playerSkills['Radi_meanTrueSkillmu']
X['Radi_meanTrueSkillsigma'] = playerSkills['Radi_meanTrueSkillsigma']

In [7]:
X = X.values

In [9]:
Y = pd.DataFrame(playerSkills['Radi_win'])
Y = Y.values

In [10]:
# So now that we have these X and Y ready to go, we can get started with the total process.
X

array([[25.21466116,  6.5180607 , 24.99506614,  6.12051626],
       [25.25767538,  6.67969932, 26.71221832,  8.06249729],
       [25.72116133,  7.53599049, 24.59110219,  7.95222144],
       ...,
       [30.6747832 ,  6.34965472, 23.90165202,  7.85640872],
       [25.80935994,  6.03820578, 26.32776747,  6.33685745],
       [23.57007334,  4.39916558, 25.26516682,  6.42260842]])

In [11]:
# 2/3 training, 1/3 testing
trainingLen = int(np.ceil(X.shape[0]*(2/3)))
Xtrain = X[0:trainingLen,:]
Xtest = X[trainingLen:,:]
Ytrain = Y[0:trainingLen,:]
Ytest = Y[trainingLen:,:]

In [12]:
XtrainMeans = np.mean(Xtrain,axis=0)
XtrainStds = np.std(Xtrain,axis=0)

In [13]:
XtrainNorm = (Xtrain - XtrainMeans)*1./XtrainStds ## normalize
XtrainNorm

array([[-0.09090512, -0.00404834, -0.25794848, -0.35804378],
       [-0.06186882,  0.12206439,  0.89897319,  1.16619501],
       [ 0.25100254,  0.79015481, -0.53011693,  1.07964075],
       ...,
       [-0.46908378,  1.31506998,  1.64773502,  0.21535685],
       [ 1.3384113 , -1.63457597,  1.12026356, -0.76113133],
       [-0.41248459,  0.19976711, -0.82027765,  0.92366979]])

In [14]:
XtestNorm = (Xtest - XtrainMeans)*1./XtrainStds ## normalize
XtestNorm

array([[-1.39131924,  0.34787375,  0.95183378, -1.32514371],
       [ 1.24738136,  0.85845012, -0.25462432,  1.37877114],
       [-0.34258088,  1.27299944, -0.90105093, -0.15334658],
       ...,
       [ 3.5948934 , -0.13544105, -0.99463012,  1.00443844],
       [ 0.3105401 , -0.3784379 ,  0.63995158, -0.18824004],
       [-1.201067  , -1.65724013, -0.07596966, -0.12093508]])

In [15]:
testingRuns = []

In [16]:
### running the regression with multiple testing samples
allTols = [.0001]
for inputTol in allTols:
    logreg = sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=inputTol, \
                                                     C=1.0,fit_intercept=True,intercept_scaling=1, \
                                                     class_weight='balanced',random_state=0, \
                                                     solver='liblinear',max_iter=100,multi_class='ovr', \
                                                     verbose=0,warm_start=False,n_jobs=None)
    logreg.fit(Xtrain,np.ravel(Ytrain))
    sc = logreg.score(Xtrain,np.ravel(Ytrain))
    p = logreg.predict(Xtest)
    results = pd.DataFrame(Ytest)
    results.rename(columns={0:'Ytest'},inplace=True)
    results['Output'] = p.T
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    
    for idx,row in results.iterrows():
        if row['Ytest']==1:
            if row['Ytest']==row['Output']:
                TP = TP + 1
            else:
                FP = FP + 1
        else:
            if row['Ytest']==row['Output']:
                TN = TN + 1
            else:
                FN = FN + 1
    
    precision = TP/(float(TP)+FP)
    recall = TP/(float(TP)+FN)
    fMeasure = (2*precision*recall)/(precision+recall)
    accuracy = (TP+TN)/(float(TP)+TN+FP+FN)
    print("Coefficient: ",logreg.coef_)
    print("Intercept: ", logreg.intercept_)
    print("Classes: ",logreg.classes_)
    print("Precision: ",precision)
    print("Recall: ",recall)
    print("fMeasure: ",fMeasure)
    print("Accuracy: ",accuracy)
    testingRuns.append((logreg,(precision,recall,fMeasure,accuracy)))

Coefficient:  [[-0.70434053 -0.02324042  0.73551793  0.03208093]]
Intercept:  [-0.83755776]
Classes:  [0 1]
Precision:  0.7583352762881791
Recall:  0.7698224852071006
fMeasure:  0.7640357058961711
Accuracy:  0.7589103564142565
